In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklego.metrics import equal_opportunity_score
from sklego.metrics import p_percent_score
from sklearn.metrics import log_loss
from sklearn.utils.extmath import squared_norm
from moopt.scalarization_interface import scalar_interface, single_interface, w_interface
from moopt import monise
import numpy as np
import optuna, sklearn, sklearn.datasets
from fair_models import coefficient_of_variation, MOOLogisticRegression, FindCLogisticRegression, FindCCLogisticRegression
from fair_models import calc_reweight
from fair_models import FairScalarization, EqualScalarization
from sklearn.model_selection import KFold

In [67]:
def kfold_methods(dataset, fair_feature, pred_feature, val_size = 100, n_folds = 5):
    kfold = KFold(n_folds, True, 1)
    
    metrics = ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation']
    methods = ['LogReg', 'RegEqual', 'RegDemo', 'RegRewe', 'RegMoo', 'RegEqMoo']
    
    index = [sum([[method]*len(metrics) for method in methods],[]),
        sum([metrics for method in methods],[])]
    results_test = pd.DataFrame(index=index, columns=metrics)
    
    for metric in metrics:
        for method in methods:
            for i in range(4):
                results_test.loc[(method,metric),metrics[i]] = 0
    
    for i in range(n_folds):
    
        result = next(kfold.split(dataset), None)

        train = dataset.iloc[result[0]].drop("Unnamed: 0", axis=1)
        test = dataset.iloc[result[1]].drop("Unnamed: 0", axis=1)

        X = train.drop([pred_feature], axis=1)
        y = train[pred_feature]
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size)
        X_test = test.drop([pred_feature], axis=1)
        y_test = test[pred_feature]

        sample_weight = calc_reweight(X_train, y_train, fair_feature)

        fair_scalarization = FairScalarization(X_train, y_train, fair_feature)
        fair_moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, fair_scalarization)
        fair_moo_learn.tune()

        equal_scalarization = EqualScalarization(X_train, y_train, fair_feature)
        equal_moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, equal_scalarization)
        equal_moo_learn.tune()

        for metric in metrics:

            # RegLog
            reg_log = FindCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, metric=metric).tune()
            results_test.loc[('LogReg',metric),metrics[0]] += reg_log.score(X_test, y_test)/n_folds
            results_test.loc[('LogReg',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_log,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('LogReg',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_log, X_test)/n_folds
            results_test.loc[('LogReg',metric),metrics[3]] += coefficient_of_variation(reg_log, X_test, y_test)/n_folds

            # RegEqual
            reg_equal = FindCCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, metric=metric, 
                                                 base_model='equal').tune()
            results_test.loc[('RegEqual',metric),metrics[0]] += reg_equal.score(X_test, y_test)/n_folds
            results_test.loc[('RegEqual',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_equal,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('RegEqual',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_equal, X_test)/n_folds
            results_test.loc[('RegEqual',metric),metrics[3]] += coefficient_of_variation(reg_equal, X_test, y_test)/n_folds

            # RegDemo
            reg_demo = FindCCLogisticRegression(X_train, y_train, X_val, y_val,fair_feature, metric=metric, 
                                                base_model='demographic').tune()
            results_test.loc[('RegDemo',metric),metrics[0]] += reg_demo.score(X_test, y_test)/n_folds
            results_test.loc[('RegDemo',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_demo,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('RegDemo',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_demo, X_test)/n_folds
            results_test.loc[('RegDemo',metric),metrics[3]] += coefficient_of_variation(reg_demo, X_test, y_test)/n_folds

            # RegRewe
            reg_rewe = FindCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature,metric=metric, 
                                               sample_weight=sample_weight).tune()
            results_test.loc[('RegRewe',metric),metrics[0]] += reg_rewe.score(X_test, y_test)/n_folds
            results_test.loc[('RegRewe',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_rewe,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('RegRewe',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_rewe, X_test)/n_folds
            results_test.loc[('RegRewe',metric),metrics[3]] += coefficient_of_variation(reg_rewe, X_test, y_test)/n_folds

            # RegMoo
            reg_moo = fair_moo_learn.tune(metric)
            results_test.loc[('RegMoo',metric),metrics[0]] += reg_moo.score(X_test, y_test)/n_folds
            results_test.loc[('RegMoo',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_moo,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('RegMoo',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_moo, X_test)/n_folds
            results_test.loc[('RegMoo',metric),metrics[3]] += coefficient_of_variation(reg_moo, X_test, y_test)/n_folds

            # RegEqMoo
            reg_eqmoo = equal_moo_learn.tune(metric)
            results_test.loc[('RegEqMoo',metric),metrics[0]] += reg_eqmoo.score(X_test, y_test)/n_folds
            results_test.loc[('RegEqMoo',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_eqmoo,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('RegEqMoo',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_eqmoo, X_test)/n_folds
            results_test.loc[('RegEqMoo',metric),metrics[3]] += coefficient_of_variation(reg_eqmoo, X_test, y_test)/n_folds

    return results_test

In [73]:
def kfold_ensemble(dataset, fair_feature, pred_feature, val_size = 100, n_folds = 5):
    kfold = KFold(n_folds, True, 1)
    
    metrics = ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation']
    ensembles = ['voting soft', 'voting hard', 'knorau', 'knorae']
    methods = ['RegMoo', 'RegEqMoo']
    
    index = [sum([[method]*len(ensembles) for method in methods],[]),
            sum([ensembles for method in methods],[])]
    results_test = pd.DataFrame(index=index, columns=metrics)
    
    for ensemble in ensembles:
        for method in methods:
            for i in range(len(metrics)):
                results_test.loc[(method,ensemble),metrics[i]] = 0
    
    for i in range(n_folds):
    
        result = next(kfold.split(dataset), None)

        train = dataset.iloc[result[0]].drop("Unnamed: 0", axis=1)
        test = dataset.iloc[result[1]].drop("Unnamed: 0", axis=1)

        X = train.drop([pred_feature], axis=1)
        y = train[pred_feature]
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size)
        X_test = test.drop([pred_feature], axis=1)
        y_test = test[pred_feature]

        fair_scalarization = FairScalarization(X_train, y_train, fair_feature)
        fair_moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, fair_scalarization)
        fair_moo_learn.tune()

        equal_scalarization = EqualScalarization(X_train, y_train, fair_feature)
        equal_moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, equal_scalarization)
        equal_moo_learn.tune()

        for ensemble in ensembles:

            # RegMoo
            reg_moo = fair_moo_learn.ensemble_model(ensemble)
            results_test.loc[('RegMoo',ensemble),metrics[0]] += reg_moo.score(X_test, y_test)/n_folds
            results_test.loc[('RegMoo',ensemble),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_moo,
                                                                                                     X_test, y_test)/n_folds
            results_test.loc[('RegMoo',ensemble),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_moo,
                                                                                                               X_test)/n_folds
            results_test.loc[('RegMoo',ensemble),metrics[3]] += coefficient_of_variation(reg_moo, X_test, y_test)/n_folds

            # RegEqMoo
            reg_eqmoo = equal_moo_learn.ensemble_model(ensemble)
            results_test.loc[('RegEqMoo',ensemble),metrics[0]] += reg_eqmoo.score(X_test, y_test)/n_folds
            results_test.loc[('RegEqMoo',ensemble),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_eqmoo,
                                                                                                     X_test, y_test)/n_folds
            results_test.loc[('RegEqMoo',ensemble),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_eqmoo, 
                                                                                                                 X_test)/n_folds
            results_test.loc[('RegEqMoo',ensemble),metrics[3]] += coefficient_of_variation(reg_eqmoo, X_test, y_test)/n_folds

    return results_test

## 1. German

In [57]:
german_data = pd.read_csv("Datasets - Limpos/german.csv")
german_fair_feature = "Sex"

In [68]:
german_result_metrics = kfold_methods(german_data, "Sex", "Risk")

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/pyt

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

...

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered 

In [69]:
german_result_metrics

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy            0.59619          0.881732  0.837138    0.842382
         equal_opportunity  0.590476           0.88392  0.839122    0.751382
         p_percent          0.590476          0.889973  0.907642    0.770161
         c_variation        0.573333          0.984589  0.985182    0.566732
RegEqual accuracy           0.609524          0.880847  0.816545    0.820122
         equal_opportunity  0.580952          0.937143  0.964838    0.682935
         p_percent          0.580952          0.885714  0.970207    0.749817
         c_variation        0.571429          0.956693  0.971335    0.600283
RegDemo  accuracy           0.590476          0.828174     0.757    0.881578
         equal_opportunity  0.577143          0.971429  0.992308    0.542577
         p_percent          0.571429                 1         1    0.489898
         c_variation        0.571429                 1         1    0.489898
RegRewe  accuracy           0.605714          0.894845  0.849523    0.826829
         equal_opportunity  0.592381          0.931825  0.888312    0.820742
         p_percent          0.605714           0.89595  0.934865    0.816706
         c_variation        0.573333          0.984589  0.985182    0.566732
RegMoo   accuracy            0.60381            0.8711  0.844508    0.841992
         equal_opportunity  0.607619          0.908281  0.808359    0.851606
         p_percent          0.594286          0.905042  0.909016    0.825173
         c_variation        0.586667          0.765671  0.738323    0.696436
RegEqMoo accuracy           0.592381           0.89026  0.929772    0.724869
         equal_opportunity  0.580952          0.989062  0.973913    0.556347
         p_percent          0.571429                 1         1    0.489898
         c_variation        0.571429                 1         1    0.489898

In [79]:
german_result_ensemble = kfold_ensemble(german_data, "Sex", "Risk")

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/pyt

In [80]:
german_result_ensemble

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft  0.601905          0.955066  0.857936    0.817419
         voting hard  0.588571          0.954552  0.908897    0.833107
         knorau       0.594286           0.96435  0.895576    0.832333
         knorae        0.55619          0.741677  0.638301    0.991349
RegEqMoo voting soft  0.571429                 1         1    0.489898
         voting hard  0.571429                 1         1    0.489898
         knorau       0.571429          0.984545  0.984938    0.520127
         knorae       0.561905          0.743043   0.63715    0.958173

## 2. LSAC

In [70]:
lsac_data = pd.read_csv("Datasets - Limpos/lsac.csv")
lsac_fair_feature = "race1"
lsac_pred_feature = "pass_bar"

In [71]:
lsac_result_metrics = kfold_methods(lsac_data, lsac_fair_feature, lsac_pred_feature, 4000)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, 

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encoun

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: Runtim

In [72]:
lsac_result_metrics

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy           0.953598          0.998513   0.99361    0.284511
         equal_opportunity   0.95301                 1         1     0.28584
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953304          0.999257  0.996805    0.285177
RegEqual accuracy           0.953647          0.999257  0.995699    0.284269
         equal_opportunity   0.95301                 1         1     0.28584
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953451          0.999257    0.9966    0.284787
RegDemo  accuracy           0.953696          0.998885  0.995117    0.284194
         equal_opportunity   0.95301                 1         1     0.28584
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953304          0.999628  0.997821     0.28512
RegRewe  accuracy           0.953304                 1  0.999653    0.285069
         equal_opportunity  0.953108                 1  0.999884    0.285583
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953304                 1  0.999653    0.285069
RegMoo   accuracy           0.953793          0.997398  0.990792    0.284159
         equal_opportunity  0.953108                 1  0.999361    0.285583
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953451          0.999257  0.996223    0.284787
RegEqMoo accuracy           0.953451          0.998513  0.994569    0.284897
         equal_opportunity   0.95301                 1         1     0.28584
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953206          0.999257  0.997444    0.285434

In [81]:
lsac_result_ensemble = kfold_ensemble(lsac_data, lsac_fair_feature, lsac_pred_feature, 4000)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib

In [82]:
lsac_result_ensemble

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft   0.95301                 1         1     0.28584
         voting hard  0.953059                 1  0.999681    0.285711
         knorau       0.953157          0.998885  0.997125    0.285622
         knorae       0.918013          0.903266  0.883078    0.405791
RegEqMoo voting soft   0.95301                 1         1     0.28584
         voting hard   0.95301                 1         1     0.28584
         knorau       0.953157                 1  0.999042    0.285454
         knorae       0.917817          0.901731   0.88176    0.406343

## 3. Adult

In [74]:
adult_data = pd.read_csv("Datasets - Limpos/adult.csv")
adult_fair_feature = "race"
adult_pred_feature = "income"

In [75]:
adult_result_metrics = kfold_methods(adult_data, adult_fair_feature, adult_pred_feature, 5000)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/p

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encounter

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encounter

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encounter

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encounter

...

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeW

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeW

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeW

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encounter

In [77]:
adult_result_metrics

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy           0.800129          0.902919  0.580725    0.702279
         equal_opportunity  0.796671          0.835233  0.507027    0.708816
         p_percent           0.79499           0.91171  0.607574    0.711518
         c_variation         0.79777          0.875411  0.545714    0.704447
RegEqual accuracy           0.833678          0.684414  0.407508    0.606748
         equal_opportunity  0.823691          0.577969  0.348672    0.642833
         p_percent          0.818035          0.631938  0.395307    0.659649
         c_variation        0.833613          0.681452  0.403632    0.603378
RegDemo  accuracy           0.833775          0.691951  0.414395     0.60538
         equal_opportunity  0.822818          0.753691   0.52616    0.637118
         p_percent          0.789593          0.942627  0.842684    0.714802
         c_variation        0.833452           0.68615  0.405625    0.603752
RegRewe  accuracy            0.79554          0.875553  0.537218    0.706152
         equal_opportunity  0.795249          0.859786  0.556886    0.709942
         p_percent          0.795281           0.87802  0.582446    0.709725
         c_variation         0.79457          0.852021  0.534675    0.706681
RegMoo   accuracy           0.799483          0.952546  0.619571    0.706299
         equal_opportunity  0.795572          0.917806  0.594572    0.711143
         p_percent          0.795992          0.915653  0.574203    0.710996
         c_variation        0.796477          0.894589  0.583932    0.707537
RegEqMoo accuracy           0.795443          0.878019  0.582497    0.708879
         equal_opportunity  0.241435                 1         1    0.344154
         p_percent          0.241435                 1         1    0.344154
         c_variation        0.241435                 1         1    0.344154

In [83]:
adult_result_ensemble = kfold_ensemble(adult_data, adult_fair_feature, adult_pred_feature, 5000)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/p

In [84]:
adult_result_ensemble

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft  0.795895          0.900096  0.556878    0.712475
         voting hard  0.795507          0.913082  0.555397    0.713257
         knorau       0.795637           0.91614  0.558075    0.712947
         knorae       0.798707          0.968673  0.604382    0.707138
RegEqMoo voting soft  0.571849          0.910869  0.739753    0.568572
         voting hard  0.572107          0.908147  0.732402    0.568269
         knorau       0.693374          0.874981  0.689004    0.684818
         knorae       0.701778          0.930457  0.840158    0.763336

## 4. 